In [37]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

##############################################################################################################################
#COMMENT DEPENDING ON WHETER YOU ARE ANALYSING 25 OR 27

# s1825
file="s1825ISCO_001"
Start=7200 
End=7680

#s1827
# file="s1827_ISCO_merge.txt"
# Start=9800 
# End=10320

##############################################################################################################################
data = pd.read_csv(file,usecols =[1,2,3,4,5,6,7]) 
# Preview the first 5 lines of the loaded data 
# data.head()
data=pd.read_csv(file,usecols=[1,2,3,4,5,6,7])

data=data.to_numpy()

#remember that original time unit is in days starting from 1 Jan 1900
Time = data[:,0]
Time = (Time-Time[0])*24*3600;

#remember that original pressure units are kPa, flow units are mL/min, volume units mL
#output units a Pa, m^3/s and m^3, respectively
PressA = data[:,1]*1e3 
FlowA = data[:,2]*1e-6/60
VolA = data[:,3]*1e-6
PressB = data[:,4]*1e3
FlowB = data[:,5]*1e-6/60
VolB = data[:,6]*1e-6 

##############################################################################################################################
# FIGURE WITH ALL DATA

# %matplotlib inline 
%matplotlib qt  
fig=plt.figure(1)

ax1 = fig.add_subplot(211)
ax1.set_ylabel('pressure [Pa]')
ax1.set_title('Upstream and downstream pressure')
ax1.plot(Time,PressA,label="upstream pressure")
ax1.plot(Time,PressB,label="downstream pressure")
plt.legend(loc="best")

#check volumes for eventual leaks

ax2 = fig.add_subplot(212)
ax2.set_ylabel('volume [m^3]')
ax2.set_title('Upstream volume')
ax2.set_xlabel('time (s)')
ax2.plot(Time,VolA)
# plt.plot(Time,VolB)

PressA=PressA[np.where(np.logical_and(Time>=Start, Time<=End))]
PressB=PressB[np.where(np.logical_and(Time>=Start, Time<=End))]
FlowA=FlowA[np.where(np.logical_and(Time>=Start, Time<=End))]
FlowB=FlowB[np.where(np.logical_and(Time>=Start, Time<=End))]
VolA=VolA[np.where(np.logical_and(Time>=Start, Time<=End))]
VolB=VolB[np.where(np.logical_and(Time>=Start, Time<=End))]
Time=Time[np.where(np.logical_and(Time>=Start, Time<=End))]

##############################################################################################################################
# ANOTHER FIGURE AFTER CROPPING 
fig=plt.figure(2)

ax3 = fig.add_subplot(211)
ax3.set_ylabel('pressure [Pa]')
ax3.set_title('Upstream and downstream pressure')
ax3.plot(Time,PressA,label="upstream pressure")
ax3.plot(Time,PressB,label="downstream pressure")
plt.legend(loc="best")

#check volumes for eventual leaks

ax4 = fig.add_subplot(212)
ax4.set_ylabel('volume [m^3]')
ax4.set_title('Upstream volume')
ax4.set_xlabel('time (s)')
ax4.plot(Time,VolA)
# plt.plot(Time,VolB)

In [28]:
# #test FFT

# ff = numpy.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
# Fyy = abs(numpy.fft.fft(yynoise))
# guess_freq = abs(ff[numpy.argmax(Fyy[1:])+1])
# print(guess_freq)

# plt.plot(Fyy)
# plt.plot(guess_freq,"ok")

In [29]:
# FIT A SINE TO PRESSURE DATA 
#(https://stackoverflow.com/questions/16716302/how-do-i-fit-a-sine-curve-to-my-data-with-pylab-and-numpy)

import numpy, scipy.optimize

def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"'''
    tt = numpy.array(tt)
    yy = numpy.array(yy)
    ff = numpy.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(numpy.fft.fft(yy))
    guess_freq = abs(ff[numpy.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = numpy.std(yy) * 2.**0.5
    guess_offset = numpy.mean(yy)
    guess = numpy.array([guess_amp, 2.*numpy.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  return A * numpy.sin(w*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*numpy.pi)
    fitfunc = lambda t: A * numpy.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, "fitfunc": fitfunc, "maxcov": numpy.max(pcov), "rawres": (guess,popt,pcov)}

In [31]:
import pylab as plt

tt=Time
yy=PressA

res = fit_sin(tt, yy)
print( "Amplitude[Pa]=%(amp)s, Period[s]=%(period)s, phase[rad]=%(phase)s, offset[Pa]=%(offset)s, Max. Cov.=%(maxcov)s" % res )

plt.figure(3)

plt.plot(tt,yy, "ok", label="original data")
plt.plot(tt, res["fitfunc"](tt), "r-", label="fit", linewidth=2)
plt.legend(loc="best")
plt.show()

Amplitude[Pa]=-502196.03800616215, Period[s]=59.997478782465414, phase[rad]=0.7847830691536202, offset[Pa]=3300640.7074573417, Max. Cov.=142975.49935250895


In [32]:
r_ext=50/2000
r_int=30/2000

period = 60

gain25=317377/501691
gain27=146443/502196

# I am not entirely sure if I did right with the phase shift units etc.

dphase25=-22.585-(-27.758)
dphase27=2.630-0.785

print ("s1825","A",gain25)
print ("s1827","A",gain27)

s1825 A 0.6326144977685467
s1827 A 0.2916052696556723


In [33]:
#on first execution run the following line:
#conda install conda-forge::coolprop

from CoolProp.CoolProp import PropsSI as pr

#put here the right values of pressure and volume of pump B right before the oscillations
P_1825=3.827e6 #Pa
V_1825=238+12.71  #mL in the pump chamber + pipes/valves/cylindrical volume in the stationary sample
P_1827=3.3e6 #Pa
V_1827=245.3+10.64 #mL in the pump chamber + pipes/valves/cylindrical volume in the stationary sample

# downstream reservoir storage 
betad25=pr('ISOTHERMAL_COMPRESSIBILITY','T',298,'P',P_1825,'water')*(V_1825)*1e-6 #m^3/Pa
betad27=pr('ISOTHERMAL_COMPRESSIBILITY','T',298,'P',P_1827,'water')*(V_1827)*1e-6 #m^3/Pa

visc25=pr('VISCOSITY','T',298,'P',P_1825,'water')
visc27=pr('VISCOSITY','T',298,'P',P_1827,'water')

print("s1825","beta_d",betad25,"visc",visc25)
print("s1827","beta_d",betad27,"visc",visc27)

s1825 beta_d 1.1240081930076943e-13 visc 0.0008925613516911413
s1827 beta_d 1.1490012251946974e-13 visc 0.0008926308449651368


In [34]:
import math

# Estimate of crack transmissivity kt (https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017JB014858), equation A4

#rearranged by aretusini using cross sectional area at halfway the cylinder S=(r_ext+r_int)/2*2*pi*t 
#sample length L=r_ext-r_int 
# where t=thickness and therefore thickness * permeability is the hydraulic transmissivity of the thin layer on the rock surface

# kt= 2*gain/sqrt(1-gain^2) * visc * beta_d / period * (r_ext-r_int)/(r_ext+r_int)

kt25 = 2*gain25/math.sqrt(1-gain25**2) * visc25 * betad25 / period * (r_ext-r_int)/(r_ext+r_int)

kt27 = 2*gain27/math.sqrt(1-gain27**2) * visc27 * betad27 / period * (r_ext-r_int)/(r_ext+r_int)

print("s1825",kt25)
print("s1827",kt27)

s1825 6.829086561710151e-19
s1827 2.6055771590940097e-19


In [1]:
# #TRY THIS TO SEE HOW CALCULATED BETA_D CHANGES WITH PRESSURE
# #on first execution run the following line:
# #conda install conda-forge::coolprop

# from CoolProp.CoolProp import PropsSI as pr
# import matplotlib.pyplot as plt
# import numpy as np

# # Saturation temperature of Water at 1 atm in K
# pressures=[0.1,0.5,1,1.5,2,2.5,3,3.5,5,10]
# pressures=np.array(pressures)*1e6

# betaf=[]

# for i in range(len(pressures)):
#     betaf.append(pr('ISOTHERMAL_COMPRESSIBILITY','T',298,'P',pressures[i],'water'))
    
# print(betaf)
# downstreambeta_s1825=np.array(betaf)*(12.71+200)*1e-6
# downstreambeta_s1827=np.array(betaf)*(10.64+200)*1e-6

# plt.plot(pressures,downstreambeta_s1825)
# plt.plot(pressures,downstreambeta_s1827)